In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import math

In [2]:
faults = pd.read_csv('first_348_faults_per_vehicle000 (1).csv')


In [3]:
# Transforming the idle_engine_hours to hours
faults['idle_engine_hours'] = faults['idle_engine_hours'] / 3600


In [4]:
# Calculating the active_engine_hours
faults['active_engine_hours'] = faults['engine_hours'] - faults['idle_engine_hours']
print(faults['active_engine_hours'])

0          13997.259776
1          14000.259414
2          14001.259289
3          13999.259651
4          13998.259762
               ...     
1684590      310.587361
1684591      310.087389
1684592      314.487222
1684593      314.037236
1684594      313.187236
Name: active_engine_hours, Length: 1684595, dtype: float64


In [5]:

boolly = lambda x: 1 if x == "on" else 0

faults['red_lamp_bool'] = faults['red_lamp_status'].apply(boolly)
print(faults['red_lamp_bool'].value_counts())


red_lamp_bool
0    1649860
1      34735
Name: count, dtype: int64


In [6]:
faults['vehicle_assembly_age'] = pd.to_datetime(faults['message_date_time']) - pd.to_datetime(faults['final_assembly_dt'])
faults['vehicle_assembly_age'] = faults['vehicle_assembly_age'].dt.days
print(faults['vehicle_assembly_age'])

0          1725.0
1          1725.0
2          1725.0
3          1725.0
4          1725.0
            ...  
1684590     151.0
1684591     151.0
1684592     151.0
1684593     151.0
1684594     151.0
Name: vehicle_assembly_age, Length: 1684595, dtype: float64


In [7]:
faults['vehicle_service_age'] = pd.to_datetime(faults['message_date_time']) - pd.to_datetime(faults['in_service_dt'])
faults['vehicle_service_age'] = faults['vehicle_service_age'].dt.days
print(faults['vehicle_service_age'])

0          826
1          826
2          826
3          826
4          826
          ... 
1684590    133
1684591    133
1684592    133
1684593    133
1684594    133
Name: vehicle_service_age, Length: 1684595, dtype: int32


In [8]:
selected = faults[['malfunction_lamp_status', 
                   'protect_lamp_status', 
                   'red_lamp_status', 
                   'vehicle_make', 
                   'yellow_lamp_status', 
                   'active_engine_hours', 
                   'engine_hours', 
                   'gross_combined_weight', 
                   'odometer_reading', 
                   'vehicle_assembly_age', 
                   'vehicle_service_age', 
                   'red_lamp_bool',]]
display(selected)


,malfunction_lamp_status,protect_lamp_status,red_lamp_status,vehicle_make,yellow_lamp_status,active_engine_hours,engine_hours,gross_combined_weight,odometer_reading,vehicle_assembly_age,vehicle_service_age,red_lamp_bool
0,off,off,off,VOLVO,on,13997.259776,13998.87,NaN,414416.949,1725.0,826,0
1,off,off,off,VOLVO,on,14000.259414,14001.87,NaN,414473.158,1725.0,826,0
2,off,off,off,VOLVO,on,14001.259289,14002.87,NaN,414500.206,1725.0,826,0
3,off,off,off,VOLVO,on,13999.259651,14000.87,NaN,414472.791,1725.0,826,0
4,off,off,off,VOLVO,on,13998.259762,13999.87,NaN,414447.862,1725.0,826,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1684590,off,off,off,VOLVO,on,310.587361,310.60,40124.132,11279.851,151.0,133,0
1684591,off,off,off,VOLVO,on,310.087389,310.10,40057.993,11257.469,151.0,133,0
1684592,off,off,off,VOLVO,on,314.487222,314.50,37875.417,11432.111,151.0,133,0
1684593,off,off,off,VOLVO,on,314.037236,314.05,37169.937,11410.189,151.0,133,0


In [9]:
volvor = lambda x: 1 if x == "VOLVO" else 0
selected['volvo'] = selected['vehicle_make'].apply(volvor)
print(selected['volvo'].mean())

0.6351235756962356


/var/folders/38/sjv6sgkx2rg98b8z0xk09pdr0000gn/T/ipykernel_5745/2099903806.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['volvo'] = selected['vehicle_make'].apply(volvor)


In [10]:
selected = selected.dropna()
print(selected)

        malfunction_lamp_status protect_lamp_status red_lamp_status   
696                         off                 off             off  \
697                         off                 off             off   
698                         off                 off             off   
699                         off                 off             off   
700                         off                 off             off   
...                         ...                 ...             ...   
1684590                     off                 off             off   
1684591                     off                 off             off   
1684592                     off                 off             off   
1684593                     off                 off             off   
1684594                     off                 off             off   

        vehicle_make yellow_lamp_status  active_engine_hours  engine_hours   
696            VOLVO                 on          4828.043541      482

In [11]:
X = selected[['volvo', 'active_engine_hours', 'gross_combined_weight', 'odometer_reading', 'vehicle_assembly_age']]
Y = selected['red_lamp_bool']

In [12]:
log_reg = sm.Logit(Y, X).fit()

Optimization terminated successfully.
         Current function value: 0.053017
         Iterations 9


In [13]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:          red_lamp_bool   No. Observations:               675797
Model:                          Logit   Df Residuals:                   675792
Method:                           MLE   Df Model:                            4
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.2438
Time:                        15:50:40   Log-Likelihood:                -35828.
converged:                       True   LL-Null:                       -28806.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
volvo                    -1.4431      0.023    -63.291      0.000      -1.488      -1.398
active_engine_hours    -3.02e-05   1.01e-05     -3.000      0.003   -4.99e-05   -1.05e-05
gross_co

In [14]:
X = selected[['volvo', 'engine_hours', 'gross_combined_weight', 'odometer_reading', 'vehicle_assembly_age']]
Y = selected['red_lamp_bool']

log_reg = sm.Logit(Y, X).fit()

print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.053017
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:          red_lamp_bool   No. Observations:               675797
Model:                          Logit   Df Residuals:                   675792
Method:                           MLE   Df Model:                            4
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.2438
Time:                        15:50:43   Log-Likelihood:                -35828.
converged:                       True   LL-Null:                       -28806.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
volvo                    -1.4431      0.023    -63.291      0.000      -1.488      -1.

In [15]:
X = selected[['volvo', 'engine_hours', 'gross_combined_weight', 'odometer_reading', 'vehicle_service_age']]
Y = selected['red_lamp_bool']

log_reg = sm.Logit(Y, X).fit()

print(log_reg.summary())

Optimization terminated successfully.
         Current function value: 0.052339
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:          red_lamp_bool   No. Observations:               675797
Model:                          Logit   Df Residuals:                   675792
Method:                           MLE   Df Model:                            4
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 -0.2279
Time:                        15:50:45   Log-Likelihood:                -35370.
converged:                       True   LL-Null:                       -28806.
Covariance Type:            nonrobust   LLR p-value:                     1.000
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
volvo                    -1.4155      0.022    -63.428      0.000      -1.459      -1.

In [16]:
X = selected[['volvo', 'active_engine_hours', 'gross_combined_weight', 'odometer_reading', 'vehicle_service_age']]
Y = selected['red_lamp_bool']

#X['intercept'] = 1

log_reg = sm.Logit(Y, X).fit()

print(log_reg.summary())

/var/folders/38/sjv6sgkx2rg98b8z0xk09pdr0000gn/T/ipykernel_5745/1412221441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1


Optimization terminated successfully.
         Current function value: 0.040006
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:          red_lamp_bool   No. Observations:               675797
Model:                          Logit   Df Residuals:                   675791
Method:                           MLE   Df Model:                            5
Date:                Tue, 25 Apr 2023   Pseudo R-squ.:                 0.06145
Time:                        15:50:47   Log-Likelihood:                -27036.
converged:                       True   LL-Null:                       -28806.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
volvo                     0.5587      0.047     11.831      0.000       0.466       0

In [17]:
coefs = sm.Logit(Y,X).fit().params.values
print(coefs)

Optimization terminated successfully.
         Current function value: 0.040006
         Iterations 10
[ 5.58678226e-01 -5.42694331e-05  8.23979233e-06  7.31641334e-06
  6.74154513e-05 -6.99089808e+00]


In [18]:
coefs2 = []
for coef in coefs:
    coefs2.append(math.exp(coef))

print(coefs2)

[1.7483600362597052, 0.9999457320394832, 1.0000082398262748, 1.0000073164401033, 1.0000674177238145, 0.0009202197250757503]


In [19]:
for c in coefs2:
    print((c - 1) * 100)

74.83600362597052
-0.005426796051677396
0.0008239826274847317
0.0007316440103322108
0.006741772381446687
-99.90797802749243
